In [1]:
import pandas as pd
import yfinance as yf
import xlwings as xw

def fetch_data(stock, start_date, end_date):
    data = yf.download(stock, start=start_date, end=end_date)['Adj Close']
    return data
def fetch_current_price(stock):
    data = yf.download(stock, period='1d', interval='1d')['Adj Close']
    return data.iloc[-1]
def main():
    portfolio_df = pd.read_excel('D:/LinkedIn/portfolio.xlsx')
    stocks = portfolio_df['SYMBOL'].tolist()
    quantities = portfolio_df['NUMBER'].tolist()
    total_shares = sum(quantities)
    weights = [quantity / total_shares for quantity in quantities]
    start_date = input("Enter the start date (YYYY-MM-DD): ")
    end_date = input("Enter the end date (YYYY-MM-DD): ")
    z_value = int(input("Enter the Z-value as a whole number percentage (e.g., 5 for 5%): "))
    df = pd.DataFrame()
    for i, stock in enumerate(stocks):
        data = fetch_data(stock + ".NS", start_date, end_date)
        data = data.dropna()
        daily_pct_change = data.pct_change() * 100
        df[f"{stock} Close"] = data.values
        df[f"{stock} Daily % Change"] = daily_pct_change.values
        df[f"{stock} Number of Shares"] = quantities[i]
        df[f"{stock} Weight"] = weights[i]
    df.insert(0, "Date", data.index)
    weighted_sum = sum(df[f"{stock} Daily % Change"] * weights[i] for i, stock in enumerate(stocks))
    df['Weighted Sum'] = weighted_sum
    df = df.round(2)
    df = df.dropna()
    portfolio_value = sum(fetch_current_price(stock + ".NS") * quantities[i] for i, stock in enumerate(stocks))
    output_file_path = 'D:/linkedin/var.xlsx'
    wb = xw.Book()
    sheet1 = wb.sheets['Sheet1']
    sheet1.range("A1").value = df
    filtered_df = df[['Date'] + [col for col in df.columns if 'Daily % Change' in col] + [col for col in df.columns if 'Weight' in col] + ['Weighted Sum']]
    sheet2 = wb.sheets.add(name='Sheet2', after=sheet1)
    sheet2.range("A1").value = filtered_df
    daily_change_and_weighted_sum_df = df[['Date'] + [col for col in df.columns if 'Daily % Change' in col] + ['Weighted Sum']]
    sheet3 = wb.sheets.add(name='Sheet3', after=sheet2)
    sheet3.range("A1").value = daily_change_and_weighted_sum_df
    sorted_df = df[['Date', 'Weighted Sum']].sort_values(by='Weighted Sum').reset_index(drop=True)
    sorted_df['Rank'] = sorted_df.reset_index().index + 1
    sheet4 = wb.sheets.add(name='Sheet4', after=sheet3)
    sheet4.range("A1").value = sorted_df
    n = len(sorted_df)
    x = round(n * (1 - z_value / 100))
    if x > 0:
        x_weighted_sum = sorted_df.loc[x - 1, 'Weighted Sum']
    else:
        x_weighted_sum = None
    sheet5 = wb.sheets.add(name='Sheet5', after=sheet4)
    var = (x_weighted_sum * portfolio_value) / 100
    sheet5.range("A1").value = [["Number of Days", "Z-value (%)", "Calculated Rank", "Weighted Sum at Rank ", "Portfolio Value", "VaR"]]
    sheet5.range("A2").value = [[n, z_value, x, x_weighted_sum, portfolio_value, var]]
    wb.save(output_file_path)
    sheet5_df = pd.read_excel(output_file_path, sheet_name='Sheet5')
    print("\n Var Result:")
    print(sheet5_df)
    wb.close()
    print(f"Data successfully saved to {output_file_path}")
if __name__ == "__main__":
    main()


Enter the start date (YYYY-MM-DD):  2024-06-01
Enter the end date (YYYY-MM-DD):  2024-08-22
Enter the Z-value as a whole number percentage (e.g., 5 for 5%):  95


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



 Var Result:
   Number of Days  Z-value (%)  Calculated Rank  Weighted Sum at Rank   \
0              54           95                3                  -3.19   

   Portfolio Value          VaR  
0    222503.994751 -7097.877433  
Data successfully saved to D:/linkedin/var.xlsx
